In [1]:
from imports import*
from utils import*
from models import*
#from Train_Eval import*

In [2]:
#Now let's train the network on the graphs

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

Device: cuda


In [3]:
args = {
    'device': device,
    'heads': 2,
    'num_features' : 1,
    'hidden' : 100,
    'num_classes' : 6,
    'dropout': 0.001,
    'alpha' : 0.1,
    'lr': 0.001,
    'epochs': 100,
}

In [4]:
side = 512

In [ ]:
features_test = raw_to_tensor("CVSynth.raw", side)

labels_test = raw_to_tensor("CVSynth_Labels.raw", side)

In [ ]:
new_side = 64

In [ ]:
cloud = torch.cartesian_prod(torch.tensor(range(0, new_side)), torch.tensor(range(0, new_side)), torch.tensor(range(0, new_side)))

In [ ]:
k_neigh = 6

In [ ]:
edges = create_edges(k_neigh, cloud)

In [ ]:
x_test = torch.tensor(view_as_windows(features_test.numpy(), (64,64,64), step=28).reshape(-1,64**3,1))

y_test = torch.tensor(view_as_windows(labels_test.numpy(), (64,64,64), step=28).reshape(-1,64**3))

In [ ]:
data_list_test = []

for i in range(x_test.size()[0]):
    data_list_test.append(Data(x=x_test[i], edge_index=edges, y=y_test[i]))

test_loader = DataLoader(data_list_test, batch_size=1, shuffle=False)

In [ ]:
batch = 1

In [ ]:
class Eval_Test:

  def __init__(self, model, device):
       
    self.model = model
    self.device = device

  def eval_function(self, data_loader):

    # Sets model to train mode
    self.model.eval()

    data_loader = data_loader

    preds = torch.zeros((6,1)).cpu()

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")): #remind that tqdm draws progress bars

      model = self.model.to(device)
      
      batch = batch.to(device)

      batch_index = batch.batch

      edge_index = batch.edge_index.type(torch.LongTensor).to(device)
      
      with torch.no_grad():
        preds = torch.cat((preds,torch.nn.functional.softmax(model((batch.x).float(), edge_index, batch_index).transpose(1,0).cpu(), dim=0)),1)

      torch.cuda.empty_cache()
      del(batch)
      del(batch_index)
      del(edge_index)
      gc.collect()
      
    return preds

In [ ]:
# Define Volume Size of the whole Volume (cube Size):
Vol_Size = 512
#Define Volume Size of the input (minibatch) Volume (minibatch cube Size):
MinBatch_Vol_Size = 64
#Define Stride:
Stride = 28

#Therefore the number of Steps are:
Steps = int((Vol_Size - MinBatch_Vol_Size) / Stride + 1)

In [ ]:
plt.imshow(labels_test[0], vmin=0, vmax=5)
plt.savefig("Labels")

plt.close()

In [ ]:
for i in range(10):

  model = GNN(args['heads'], args['num_features'], args['hidden'], args['num_classes'], args['dropout']).to(device)
  model.load_state_dict(torch.load('GNN_200_non_augm_' + str(i) + '.h5'))
  model.eval()

  eval_obj = Eval_Test(model, device)


  preds = eval_obj.eval_function(test_loader)


  all_preds = preds[:,1:].reshape(6, 17, 17, 17, 64, 64, 64)


  summed_preds = torch.zeros(6, 512,512,512)


  for l in range(6):
      for i in range(Steps):
          for j in range(Steps):
              for k in range(Steps):
                  summed_preds[l,(i)*Stride:(i)*Stride+MinBatch_Vol_Size, 
                  (j)*Stride:(j)*Stride+MinBatch_Vol_Size, 
                  (k)*Stride:(k)*Stride+MinBatch_Vol_Size] = summed_preds[l,(i)*Stride:(i)*Stride+MinBatch_Vol_Size, 
                  (j)*Stride:(j)*Stride+MinBatch_Vol_Size, 
                  (k)*Stride:(k)*Stride+MinBatch_Vol_Size] + all_preds[l, i, j, k, :, :, :]


  preds_argmax = torch.argmax(summed_preds, dim=0)

  plt.imshow(preds_argmax[0], vmin=0, vmax=5)
  #plt.plot(range(len(losses)), losses)
  plt.savefig("Preds_Gnn_" + str(i))
  plt.close()

  DICE = dice(preds_argmax, labels_test, average='none', num_classes=6)

  data = [i, 100 * DICE[0], 100 * DICE[1], 100 * DICE[2],
          100 * DICE[3], 100 * DICE[4], 100 * DICE[5]]

  with open('Dice_Test_Synthetic_gnn_200_non_augm.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)